IMPORTING NECESSARY LIBRARIES

In [ ]:
#APT3010 PROJECT
#ANNLINET,  ADRIAN

from IPython import get_ipython
from IPython.display import display

# # Importing The Neccesary Libraries
import nltk
import json
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import random
import warnings
warnings.filterwarnings('ignore')
# Download the 'punkt_tab' data package
nltk.download('punkt_tab')


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

CREATE QUERY BASED INTENTS FOR DATASET

In [ ]:
{
  "intents": [
    {
      "tag": "borrow",
      "patterns": ["How do I borrow books from the library?", "How can I check out books?", "How do I borrow books with a library card?", "Can I borrow books from the library?", "What is the process to borrow books?"],
      "responses": [
        "You can borrow books by presenting your library card at the front desk or using the self-checkout machine. If you need assistance, library staff will be happy to guide you.",
        "To borrow books, just show your library card at the desk, or use the self-checkout machine available for your convenience."
      ]
    },
    {
      "tag": "renew",
      "patterns": ["How can I renew a book?", "How do I extend the due date for a book?", "Can I renew books online?", "Can I extend my book loan?"],
      "responses": [
        "Books can be renewed online via the library portal, or you can renew them in person at the library desk. You can extend your loan period if the book is not reserved by someone else.",
        "You can easily renew books through the library portal or by visiting the library desk. Ensure the book isn’t reserved by another user before renewing."
      ]
    },
    {
      "tag": "hours",
      "patterns": ["What are the library's working hours?", "When is the library open?", "What time does the library open and close?", "What are the operating hours for the library?"],
      "responses": [
        "The library is open Monday to Friday from 8 AM to 8 PM and on Saturday from 10 AM to 4 PM. We are closed on Sundays.",
        "The library operates from 8 AM to 8 PM on weekdays and 10 AM to 4 PM on Saturdays. It's closed on Sundays for maintenance."
      ]
    },
    {
      "tag": "wifi",
      "patterns": ["Does the library provide Wi-Fi access?", "Can I connect to Wi-Fi at the library?", "Is there free Wi-Fi in the library?", "How can I access the Wi-Fi at the library?"],
      "responses": [
        "Yes, the library offers free Wi-Fi for all visitors. The network name is 'Library_WiFi', and no password is required. You can connect from anywhere in the library.",
        "The library provides free Wi-Fi. Simply connect to 'Library_WiFi' and start browsing. There is no password required."
      ]
    },
    {
      "tag": "events",
      "patterns": ["What upcoming events are scheduled at the library?", "Are there any events or workshops at the library?", "Tell me about library events and activities?", "What events are happening at the library?"],
      "responses": [
        "The library hosts regular events such as book readings, workshops, and community gatherings. Check the library's event calendar on our website for the most up-to-date information.",
        "The library organizes various events, including book discussions, author talks, and workshops. For more details, visit our library's events page on the website."
      ]
    },
    {
      "tag": "computers",
      "patterns": ["Can I use computers at the library?", "Are computers available for use in the library?", "Can I access computers at the library for research?", "Does the library provide public access computers?"],
      "responses": [
        "Yes, the library provides computers for public use. You can access them with a valid library card. Make sure to book in advance during peak hours.",
        "The library has public access computers available. Just present your library card to access them."
      ]
    },
    {
      "tag": "fees",
      "patterns": ["What happens if I return a book late?", "What are the late fees for returning books?", "Is there a fine for late book returns?", "How much is the fine for overdue books?"],
      "responses": [
        "Late returns incur a fine of $0.50 per day. Please return books on time to avoid penalties. You can also renew books if needed before the due date.",
        "A fine of $0.50 per day is charged for overdue books. Be sure to return your books on time, or renew them if necessary, to avoid late fees."
      ]
    },
    {
      "tag": "reservations",
      "patterns": ["Can I reserve books that are checked out?", "How do I place a hold on a book?", "Can I reserve a book that someone else has borrowed?", "What is the procedure for reserving a book?"],
      "responses": [
        "Yes, you can place a hold on books through the library portal. You'll be notified when the book is available for pickup.",
        "You can reserve checked-out books by placing a hold via the library portal. Once available, the library will notify you to pick it up."
      ]
    },
    {
      "tag": "printing",
      "patterns": ["Does the library have printing facilities?", "Can I print documents at the library?", "Is there printing and scanning available at the library?", "Where can I print and scan documents at the library?"],
      "responses": [
        "Yes, the library offers printing and scanning services for a small fee. Please visit the help desk for more information about pricing and usage.",
        "Printing and scanning services are available in the library. You can get more details at the library help desk, and there may be a small fee for these services."
      ]
    },
    {
      "tag": "card",
      "patterns": ["How do I get a library card?", "How can I apply for a library card?", "Where can I get a library card?", "What do I need to get a library card?"],
      "responses": [
        "You can apply for a library card at the front desk by providing a valid ID and proof of address. Once your application is processed, you can start borrowing books immediately.",
        "To obtain a library card, visit the front desk with a valid ID and proof of address. You’ll receive your card after registration."
      ]
    },
    {
      "tag": "student_card",
      "patterns": ["How do I get a student library card?", "Where can I apply for a student library card?", "What is needed to get a student library card?", "How can I apply for a student card?"],
      "responses": [
        "Students can apply for a library card by providing a valid student ID and proof of enrollment at the front desk. You can start using the library services right away.",
        "To get a student library card, bring your student ID and proof of enrollment to the front desk, and you’ll receive your card on the spot."
      ]
    },
    {
      "tag": "ebooks",
      "patterns": ["Can I borrow eBooks from the library?", "Does the library offer digital books?", "How can I borrow eBooks?", "Can I access eBooks from the library portal?"],
      "responses": [
        "Yes, the library offers a selection of eBooks that can be borrowed through the library's online portal. Simply log in and browse available titles.",
        "You can borrow eBooks through the library’s online portal. Simply sign in and select from the available eBook collection."
      ]
    },
    {
      "tag": "loan_period",
      "patterns": ["How long can I borrow a book?", "What is the loan period for a book?", "How long can I keep a book from the library?", "How much time do I get to borrow a book?"],
      "responses": [
        "Books are typically borrowed for 2 weeks. You can renew them if you need more time, provided they are not reserved by others.",
        "The standard borrowing period for books is 2 weeks. If needed, you can renew the book if no one else has placed a hold on it."
      ]
    },
    {
      "tag": "return",
      "patterns": ["Can I return books at any library branch?", "Where can I return borrowed books?", "Can I return books to any branch?", "Is it possible to drop off books at any library?"],
      "responses": [
        "Yes, you can return borrowed books to any branch of the library system. You don’t need to return them to the branch where you borrowed them.",
        "Books can be returned at any branch of the library. Feel free to drop them off at any convenient location."
      ]
    },
    {
      "tag": "limit",
      "patterns": ["What is the maximum number of books I can borrow?", "How many books can I borrow at once?", "Is there a limit to how many books I can borrow?", "What’s the maximum borrow limit for books?"],
      "responses": [
        "You can borrow up to 10 books at a time. Additional items may require special permission or approval from library staff.",
        "The library allows you to borrow a maximum of 10 books at once. Please inquire at the desk if you need to borrow more."
      ]
    },
    {
      "tag": "waitlist",
      "patterns": ["Is there a waiting list for popular books?", "Can I reserve a book if it’s checked out?", "How does the waitlist for books work?", "Can I put a book on hold if it is unavailable?"],
      "responses": [
        "Yes, if a book is currently checked out, you can place a hold on it through the library portal. You’ll be notified when it’s available.",
        "If a book is unavailable, you can add yourself to the waitlist by placing a hold via the library system. You will be notified when it's ready for pickup."
      ]
    }
  ]
}


{'intents': [{'tag': 'borrow',
   'patterns': ['How do I borrow books from the library?',
    'How can I check out books?',
    'How do I borrow books with a library card?',
    'Can I borrow books from the library?',
    'What is the process to borrow books?'],
   'responses': ['You can borrow books by presenting your library card at the front desk or using the self-checkout machine. If you need assistance, library staff will be happy to guide you.',
    'To borrow books, just show your library card at the desk, or use the self-checkout machine available for your convenience.']},
  {'tag': 'renew',
   'patterns': ['How can I renew a book?',
    'How do I extend the due date for a book?',
    'Can I renew books online?',
    'Can I extend my book loan?'],
   'responses': ['Books can be renewed online via the library portal, or you can renew them in person at the library desk. You can extend your loan period if the book is not reserved by someone else.',
    'You can easily renew books 

DOWNLOADING NLP PACKAGE

In [ ]:

nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

INTENT CLASSIFICATION

In [ ]:
intents = {
  "intents": [
    {
      "tag": "library_borrowing",
      "patterns": ["How do I borrow books from the library?", "How can I check out books?", "How do I borrow books with a library card?", "Can I borrow books from the library?", "What is the process to borrow books?"],
      "responses": [
        "You can borrow books by presenting your library card at the front desk or using the self-checkout machine. If you need assistance, library staff will be happy to guide you.",
        "To borrow books, just show your library card at the desk, or use the self-checkout machine available for your convenience."
      ]
    },
    {
      "tag": "library_renewal",
      "patterns": ["How can I renew a book?", "How do I extend the due date for a book?", "Can I renew books online?", "Can I extend my book loan?"],
      "responses": [
        "Books can be renewed online via the library portal, or you can renew them in person at the library desk. You can extend your loan period if the book is not reserved by someone else.",
        "You can easily renew books through the library portal or by visiting the library desk. Ensure the book isn’t reserved by another user before renewing."
      ]
    },
    {
      "tag": "library_hours",
      "patterns": ["What are the library's working hours?", "When is the library open?", "What time does the library open and close?", "What are the operating hours for the library?"],
      "responses": [
        "The library is open Monday to Friday from 8 AM to 8 PM and on Saturday from 10 AM to 4 PM. We are closed on Sundays.",
        "The library operates from 8 AM to 8 PM on weekdays and 10 AM to 4 PM on Saturdays. It's closed on Sundays for maintenance."
      ]
    },
    {
      "tag": "library_wifi",
      "patterns": ["Does the library provide Wi-Fi access?", "Can I connect to Wi-Fi at the library?", "Is there free Wi-Fi in the library?", "How can I access the Wi-Fi at the library?"],
      "responses": [
        "Yes, the library offers free Wi-Fi for all visitors. The network name is 'Library_WiFi', and no password is required. You can connect from anywhere in the library.",
        "The library provides free Wi-Fi. Simply connect to 'Library_WiFi' and start browsing. There is no password required."
      ]
    },
    {
      "tag": "library_events",
      "patterns": ["What upcoming events are scheduled at the library?", "Are there any events or workshops at the library?", "Tell me about library events and activities?", "What events are happening at the library?"],
      "responses": [
        "The library hosts regular events such as book readings, workshops, and community gatherings. Check the library's event calendar on our website for the most up-to-date information.",
        "The library organizes various events, including book discussions, author talks, and workshops. For more details, visit our library's events page on the website."
      ]
    },
    {
      "tag": "library_computers",
      "patterns": ["Can I use computers at the library?", "Are computers available for use in the library?", "Can I access computers at the library for research?", "Does the library provide public access computers?"],
      "responses": [
        "Yes, the library provides computers for public use. You can access them with a valid library card. Make sure to book in advance during peak hours.",
        "The library has public access computers available. Just present your library card to access them."
      ]
    },
    {
      "tag": "library_late_fees",
      "patterns": ["What happens if I return a book late?", "What are the late fees for returning books?", "Is there a fine for late book returns?", "How much is the fine for overdue books?"],
      "responses": [
        "Late returns incur a fine of $0.50 per day. Please return books on time to avoid penalties. You can also renew books if needed before the due date.",
        "A fine of $0.50 per day is charged for overdue books. Be sure to return your books on time, or renew them if necessary, to avoid late fees."
      ]
    },
    {
      "tag": "library_reservations",
      "patterns": ["Can I reserve books that are checked out?", "How do I place a hold on a book?", "Can I reserve a book that someone else has borrowed?", "What is the procedure for reserving a book?"],
      "responses": [
        "Yes, you can place a hold on books through the library portal. You'll be notified when the book is available for pickup.",
        "You can reserve checked-out books by placing a hold via the library portal. Once available, the library will notify you to pick it up."
      ]
    },
    {
      "tag": "library_printing_scanning",
      "patterns": ["Does the library have printing facilities?", "Can I print documents at the library?", "Is there printing and scanning available at the library?", "Where can I print and scan documents at the library?"],
      "responses": [
        "Yes, the library offers printing and scanning services for a small fee. Please visit the help desk for more information about pricing and usage.",
        "Printing and scanning services are available in the library. You can get more details at the library help desk, and there may be a small fee for these services."
      ]
    },
    {
      "tag": "library_card",
      "patterns": ["How do I get a library card?", "How can I apply for a library card?", "Where can I get a library card?", "What do I need to get a library card?"],
      "responses": [
        "You can apply for a library card at the front desk by providing a valid ID and proof of address. Once your application is processed, you can start borrowing books immediately.",
        "To obtain a library card, visit the front desk with a valid ID and proof of address. You’ll receive your card after registration."
      ]
    },
    {
      "tag": "student_library_card",
      "patterns": ["How do I get a student library card?", "Where can I apply for a student library card?", "What is needed to get a student library card?", "How can I apply for a student card?"],
      "responses": [
        "Students can apply for a library card by providing a valid student ID and proof of enrollment at the front desk. You can start using the library services right away.",
        "To get a student library card, bring your student ID and proof of enrollment to the front desk, and you’ll receive your card on the spot."
      ]
    },
    {
      "tag": "library_fees",
      "patterns": ["Are there any fees for library services?", "Does the library charge for services?", "Are there charges for using the library?", "What are the costs for library services?"],
      "responses": [
        "Most library services are free, but there are small fees for printing, scanning, and late book returns. The fees are outlined on the library's website.",
        "Library services such as borrowing books, reading, and using study areas are free, but charges apply for printing, scanning, and overdue books."
      ]
    },
    {
      "tag": "library_ebooks",
      "patterns": ["Can I borrow eBooks from the library?", "Does the library offer digital books?", "How can I borrow eBooks?", "Can I access eBooks from the library portal?"],
      "responses": [
        "Yes, the library offers a selection of eBooks that can be borrowed through the library's online portal. Simply log in and browse available titles.",
        "You can borrow eBooks through the library’s online portal. Simply sign in and select from the available eBook collection."
      ]
    },
    {
      "tag": "library_loan_period",
      "patterns": ["How long can I borrow a book?", "What is the loan period for a book?", "How long can I keep a book from the library?", "How much time do I get to borrow a book?"],
      "responses": [
        "Books are typically borrowed for 2 weeks. You can renew them if you need more time, provided they are not reserved by others.",
        "The standard borrowing period for books is 2 weeks. If needed, you can renew the book if no one else has placed a hold on it."
      ]
    },
    {
      "tag": "library_return",
      "patterns": ["Can I return books at any library branch?", "Where can I return borrowed books?", "Can I return books to any branch?", "Is it possible to drop off books at any library?"],
      "responses": [
        "Yes, you can return borrowed books to any branch of the library system. You don’t need to return them to the branch where you borrowed them.",
        "Books can be returned at any branch of the library. Feel free to drop them off at any convenient location."
      ]
    },
    {
      "tag": "library_borrow_limit",
      "patterns": ["What is the maximum number of books I can borrow?", "How many books can I borrow at once?", "Is there a limit to how many books I can borrow?", "What’s the maximum borrow limit for books?"],
      "responses": [
        "You can borrow up to 10 books at a time. Additional items may require special permission or approval from library staff.",
        "The library allows you to borrow a maximum of 10 books at once. Please inquire at the desk if you need to borrow more."
      ]
    },
    {
      "tag": "library_waitlist",
      "patterns": ["Is there a waiting list for popular books?", "Can I reserve a book if it’s checked out?", "How does the waitlist for books work?", "Can I put a book on hold if it is unavailable?"],
      "responses": [
        "Yes, if a book is currently checked out, you can place a hold on it through the library portal. You’ll be notified when it’s available.",
        "If a book is unavailable, you can add yourself to the waitlist via the library's online system and be notified when it becomes available."
      ]
    },
    {
      "tag": "library_donations",
      "patterns": ["Can I donate books to the library?", "Does the library accept book donations?", "How can I donate my books to the library?", "Can I give my books to the library?"],
      "responses": [
        "Yes, the library accepts book donations. Please contact the library to arrange for donations, and follow any specific guidelines they have.",
        "The library is happy to receive book donations. You can contact the library for donation guidelines and instructions."
      ]
    },
    {
      "tag": "library_study_area",
      "patterns": ["Does the library have study rooms?", "Is there a place to study quietly?", "Can I study in a quiet area?", "Where can I study in the library?"],
      "responses": [
        "Yes, the library offers quiet study areas for individual work. Please be mindful of others when using the study spaces.",
        "The library has designated quiet study areas where you can work in peace without distractions."
      ]
    },
    {
      "tag": "library_event_hosting",
      "patterns": ["Can I host an event at the library?", "Can I rent space for events at the library?", "How can I organize an event at the library?", "Is it possible to book the library for an event?"],
      "responses": [
        "Yes, the library offers spaces that can be reserved for events. Please contact the library to inquire about availability, booking process, and any associated fees.",
        "You can host an event at the library. Contact the library’s event coordinator for more details on booking spaces and event requirements."
      ]
    },
    {
      "tag": "lost_library_card",
      "patterns": ["I lost my library card, what should I do?", "What happens if I lose my library card?", "How do I replace a lost library card?", "I can’t find my library card, what should I do?"],
      "responses": [
        "If your library card is lost or stolen, report it immediately at the library desk or through the library’s online portal to avoid unauthorized use.",
        "To report a lost library card, please visit the library desk. A new card will be issued after verification."
      ]
    }
  ]
}


stemmer = WordNetLemmatizer()

words = []
classes = []
documents = []
ignore_words = ['?']


for intent in intents['intents']: # Now 'intents' is defined and accessible
    for pattern in intent['patterns']:
        # Tokenize each word in the sentence
        w = nltk.word_tokenize(pattern)
        # Add to our words list
        words.extend(w)
        # Add to documents in our corpus
        documents.append((w, intent['tag']))
        # Add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])


words = [stemmer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))


classes = sorted(list(set(classes)))

print(len(documents), "documents")
print(len(classes), "classes", classes)
print(len(words), "unique lemmatized words", words)

85 documents
21 classes ['library_borrow_limit', 'library_borrowing', 'library_card', 'library_computers', 'library_donations', 'library_ebooks', 'library_event_hosting', 'library_events', 'library_fees', 'library_hours', 'library_late_fees', 'library_loan_period', 'library_printing_scanning', 'library_renewal', 'library_reservations', 'library_return', 'library_study_area', 'library_waitlist', 'library_wifi', 'lost_library_card', 'student_library_card']
147 unique lemmatized words ["'s", ',', 'a', 'about', 'accept', 'access', 'activity', 'an', 'and', 'any', 'apply', 'are', 'area', 'at', 'available', 'book', 'borrow', 'borrowed', 'branch', 'can', 'card', 'charge', 'check', 'checked', 'close', 'computer', 'connect', 'cost', 'date', 'digital', 'do', 'document', 'doe', 'donate', 'donation', 'drop', 'due', 'ebooks', 'else', 'event', 'extend', 'facility', 'fee', 'find', 'fine', 'for', 'free', 'from', 'get', 'give', 'ha', 'happening', 'happens', 'have', 'hold', 'host', 'hour', 'how', 'i', 'i

In [ ]:
training = []
output = []
output_empty = [0] * len(classes)

# Training set, bag of words for each sentence
for doc in documents:
    # Initialize our bag of words
    bag = []
    # List of tokenized words for the pattern
    pattern_words = doc[0]
    # Lemmatize each word
    pattern_words = [stemmer.lemmatize(word.lower()) for word in pattern_words]
    # Create our bag of words array
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # Output is a '0' for each tag and '1' for the current tag
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

random.shuffle(training)


def synonym_replacement(tokens, limit):
    augmented_sentences = []
    for i in range(len(tokens)):
        synonyms = []
        for syn in wordnet.synsets(tokens[i]):
            for lemma in syn.lemmas():
                synonyms.append(lemma.name())
        if len(synonyms) > 0:
            num_augmentations = min(limit, len(synonyms))
            sampled_synonyms = random.sample(synonyms, num_augmentations)
            for synonym in sampled_synonyms:
                augmented_tokens = tokens[:i] + [synonym] + tokens[i + 1:]
                augmented_sentences.append(' '.join(augmented_tokens))
    return augmented_sentences


# Augment the training data using synonym replacement
augmented_data = []
limit_per_tag = 100

for i, doc in enumerate(training):
    bag, output_row = doc
    tokens = [words[j] for j in range(len(words)) if bag[j] == 1]
    augmented_sentences = synonym_replacement(tokens, limit_per_tag)
    for augmented_sentence in augmented_sentences:
        augmented_bag = [1 if augmented_sentence.find(word) >= 0 else 0 for word in words]
        augmented_data.append([augmented_bag, output_row])

# Convert training and augmented_data to NumPy arrays with dtype=object
training_array = np.array(training, dtype=object)
augmented_data_array = np.array(augmented_data, dtype=object)

# Concatenate the arrays
combined_data = np.concatenate((training_array, augmented_data_array), axis=0)
random.shuffle(combined_data)

SPLITTING DATASET

In [ ]:
from sklearn.model_selection import train_test_split


def separate_data_by_tags(data):
    data_by_tags = {}
    for d in data:
        tag = tuple(d[1])
        if tag not in data_by_tags:
            data_by_tags[tag] = []
        data_by_tags[tag].append(d)
    return data_by_tags.values()


separated_data = separate_data_by_tags(combined_data)

# Lists to store training and testing data
training_data = []
testing_data = []

# Split each tag's data into training and testing sets
for tag_data in separated_data:
    train_data, test_data = train_test_split(tag_data, test_size=0.2, random_state=42)
    training_data.extend(train_data)
    testing_data.extend(test_data)


random.shuffle(training_data)
random.shuffle(testing_data)

# Convert training and testing data back to np.array
train_x = np.array([d[0] for d in training_data])
train_y = np.array([d[1] for d in training_data])
test_x = np.array([d[0] for d in testing_data])
test_y = np.array([d[1] for d in testing_data])

TRAINING MODEL

In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(NeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu1 = nn.ReLU()
        self.bn1 = nn.BatchNorm1d(hidden_size)
        self.dropout1 = nn.Dropout(0.2)

        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.relu2 = nn.ReLU()
        self.bn2 = nn.BatchNorm1d(hidden_size)
        self.dropout2 = nn.Dropout(0.2)

        self.fc3 = nn.Linear(hidden_size, output_size)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.bn1(x)
        x = self.dropout1(x)

        x = self.fc2(x)
        x = self.relu2(x)
        x = self.bn2(x)
        x = self.dropout2(x)

        x = self.fc3(x)
        output = self.softmax(x)
        return output

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return self.softmax(x)

class CustomDataset(Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]

def accuracy(predictions, targets):
    predicted_labels = torch.argmax(predictions, dim=1)
    true_labels = torch.argmax(targets, dim=1)
    correct = (predicted_labels == true_labels).sum().item()
    total = targets.size(0)
    return correct / total

def test_model(model, test_loader, criterion):
    model.eval()
    total_loss = 0.0
    total_accuracy = 0.0
    num_batches = len(test_loader)

    with torch.no_grad():
        for inputs, targets in test_loader:
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            total_loss += loss.item() * inputs.size(0)
            total_accuracy += accuracy(outputs, targets) * inputs.size(0)

    average_loss = total_loss / len(test_loader.dataset)
    average_accuracy = total_accuracy / len(test_loader.dataset)
    return average_loss, average_accuracy

In [ ]:
# Create DataLoader for training and testing data
train_x = torch.tensor(train_x).float()
train_y = torch.tensor(train_y).float()
test_x = torch.tensor(test_x).float()
test_y = torch.tensor(test_y).float()

batch_size = 64
train_dataset = CustomDataset(train_x, train_y)
test_dataset = CustomDataset(test_x, test_y)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Define the model, loss function, and optimizer
input_size = len(train_x[0])
hidden_size = 8
output_size = len(train_y[0])
model = NeuralNetwork(input_size, hidden_size, output_size)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters())

# Train the model and evaluate on the testing set
num_epochs = 50
for epoch in range(num_epochs):
    # Training
    model.train()
    running_loss = 0.0
    running_acc = 0.0
    for inputs, targets in train_loader:
        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, targets)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Update statistics
        running_loss += loss.item() * inputs.size(0)
        running_acc += accuracy(outputs, targets) * inputs.size(0)

    # Calculate average training loss and accuracy for the epoch
    epoch_loss = running_loss / len(train_loader.dataset)
    epoch_acc = running_acc / len(train_loader.dataset)

    # Print training loss and accuracy for each epoch
    print(f"Epoch [{epoch+1}/{num_epochs}], Training Loss: {epoch_loss:.4f}, Training Accuracy: {epoch_acc:.4f}")

    # Evaluate on the testing set
    test_loss, test_accuracy = test_model(model, test_loader, criterion)
    print(f"Epoch [{epoch+1}/{num_epochs}], Testing Loss: {test_loss:.4f}, Testing Accuracy: {test_accuracy:.4f}")

# Save the trained model
torch.save(model.state_dict(), 'model.pth')

Epoch [1/50], Training Loss: 0.1722, Training Accuracy: 0.2086
Epoch [1/50], Testing Loss: 0.1420, Testing Accuracy: 0.5118
Epoch [2/50], Training Loss: 0.1091, Training Accuracy: 0.6557
Epoch [2/50], Testing Loss: 0.0766, Testing Accuracy: 0.7353
Epoch [3/50], Training Loss: 0.0536, Training Accuracy: 0.8501
Epoch [3/50], Testing Loss: 0.0354, Testing Accuracy: 0.9179
Epoch [4/50], Training Loss: 0.0253, Training Accuracy: 0.9453
Epoch [4/50], Testing Loss: 0.0185, Testing Accuracy: 0.9614
Epoch [5/50], Training Loss: 0.0141, Training Accuracy: 0.9685
Epoch [5/50], Testing Loss: 0.0117, Testing Accuracy: 0.9759
Epoch [6/50], Training Loss: 0.0092, Training Accuracy: 0.9814
Epoch [6/50], Testing Loss: 0.0082, Testing Accuracy: 0.9843
Epoch [7/50], Training Loss: 0.0065, Training Accuracy: 0.9883
Epoch [7/50], Testing Loss: 0.0061, Testing Accuracy: 0.9893
Epoch [8/50], Training Loss: 0.0048, Training Accuracy: 0.9911
Epoch [8/50], Testing Loss: 0.0048, Testing Accuracy: 0.9931
Epoch [9

MODEL INFERENCE


In [ ]:
def load_model(model_path, input_size, hidden_size, output_size):
    model = NeuralNetwork(input_size, hidden_size, output_size)
    model.load_state_dict(torch.load(model_path))
    model.eval()
    return model

# Function to preprocess the input sentence
def preprocess_sentence(sentence, words):
    sentence_words = sentence.lower().split()
    sentence_words = [word for word in sentence_words if word in words]
    return sentence_words

# Function to convert the preprocessed sentence into a feature vector
def sentence_to_features(sentence_words, words):
    features = [1 if word in sentence_words else 0 for word in words]
    return torch.tensor(features).float().unsqueeze(0)

# Function to generate a response using the trained model
def generate_response(sentence, model, words, classes):
    sentence_words = preprocess_sentence(sentence, words)
    if len(sentence_words) == 0:
        return "I'm sorry, but I don't understand. Can you please rephrase or provide more information?"

    features = sentence_to_features(sentence_words, words)
    with torch.no_grad():
        outputs = model(features)

    probabilities, predicted_class = torch.max(outputs, dim=1)
    confidence = probabilities.item()
    predicted_tag = classes[predicted_class.item()]

    if confidence > 0.5:
        for intent in intents['intents']:
            if intent['tag'] == predicted_tag:
                return random.choice(intent['responses'])

    return "I'm sorry, but I'm not sure how to respond to that."

In [ ]:
model_path = 'model.pth'
input_size = len(words)
hidden_size = 8
output_size = len(classes)
model = load_model(model_path, input_size, hidden_size, output_size)

# Test the chatbot response
print('Hello! I am a chatbot. How can I help you today? Type "quit" to exit.')
while True:
    user_input = input('> ')
    if user_input.lower() == 'quit':
        break
    response = generate_response(user_input, model, words, classes)
    print(response)

Hello! I am a chatbot. How can I help you today? Type "quit" to exit.
> renew
Books can be renewed online via the library portal, or you can renew them in person at the library desk. You can extend your loan period if the book is not reserved by someone else.
> card
You can apply for a library card at the front desk by providing a valid ID and proof of address. Once your application is processed, you can start borrowing books immediately.
> hours
I'm sorry, but I don't understand. Can you please rephrase or provide more information?
> loan
You can easily renew books through the library portal or by visiting the library desk. Ensure the book isn’t reserved by another user before renewing.
> quit


In [ ]:
import textwrap

# Data for questions, answers, and tags
library_data = [
    {
        "question": "How do I borrow books from the library?",
        "answer": "You can borrow books by presenting your library card at the front desk or using the self-checkout machine.",
        "tags": ["borrow", "checkout", "library card", "loan", "get books"]
    },
    {
        "question": "How can I renew a book?",
        "answer": "Books can be renewed online via the library portal or in person at the library desk.",
        "tags": ["renew", "renewal", "library portal", "extend due date", "book renewal"]
    },
    {
        "question": "What are the library's working hours?",
        "answer": "The library is open Monday to Friday, 8 AM to 8 PM, and Saturday, 10 AM to 4 PM.",
        "tags": ["hours", "open", "closing time", "schedule", "timings", "when open"]
    },
    {
        "question": "Does the library provide Wi-Fi access?",
        "answer": "Yes, the library offers free Wi-Fi for all visitors. The network name is 'Library_WiFi', and no password is required.",
        "tags": ["wifi", "internet", "access", "wireless", "connect to Wi-Fi", "online access"]
    },
    {
        "question": "What upcoming events are scheduled at the library?",
        "answer": "The library hosts regular events such as book readings, workshops, and community gatherings. Check the library's event calendar on our website for details.",
        "tags": ["event", "events", "workshop", "calendar", "activities", "happenings"]
    },
    {
        "question": "Can I use computers at the library?",
        "answer": "Yes, the library provides computers for public use. You can access them with a valid library card.",
        "tags": ["computer", "public access", "library computers", "PC", "technology", "use computers"]
    },
    {
        "question": "What happens if I return a book late?",
        "answer": "Late returns incur a fine of $0.50 per day. Please return books on time to avoid penalties.",
        "tags": ["late", "fine", "return", "overdue", "late fees"]
    },
    {
        "question": "Can I reserve books that are already checked out?",
        "answer": "Yes, you can place a hold on books through the library portal, and you'll be notified when they're available.",
        "tags": ["reserve", "hold", "checked out", "waiting list", "book reservation"]
    },
    {
        "question": "Does the library have printing and scanning facilities?",
        "answer": "Yes, the library offers printing and scanning services for a small fee. Please visit the help desk for more information.",
        "tags": ["printing", "scanning", "copying", "documents", "facilities"]
    },
    {
        "question": "How do I get a library card?",
        "answer": "You can apply for a library card at the front desk by providing a valid ID and proof of address.",
        "tags": ["library card", "apply", "membership", "get card", "register"]
    },
    {
        "question": "How do I get a student library card?",
        "answer": "Students can apply for a library card by providing a valid student ID and proof of enrollment at the front desk.",
        "tags": ["student card", "membership", "student ID", "register", "student library"]
    },
    {
        "question": "Are there any fees for library services?",
        "answer": "Most library services are free, but there are small fees for printing, scanning, and late book returns.",
        "tags": ["fees", "cost", "printing", "scanning", "late returns"]
    },
    {
        "question": "Can I borrow eBooks from the library?",
        "answer": "Yes, the library offers a selection of eBooks that can be borrowed through the library's online portal.",
        "tags": ["ebooks", "digital books", "borrow online", "library portal", "eBooks"]
    },
    {
        "question": "How long can I borrow a book?",
        "answer": "Books are typically borrowed for 2 weeks. You can renew them if you need more time.",
        "tags": ["borrow", "duration", "loan period", "due date", "borrow time"]
    },
    {
        "question": "Can I return books at any library branch?",
        "answer": "Yes, you can return borrowed books to any branch of the library system.",
        "tags": ["return", "any branch", "drop off", "library system", "book return"]
    },
    {
        "question": "What is the maximum number of books I can borrow at once?",
        "answer": "You can borrow up to 10 books at a time. Additional items may require special permission.",
        "tags": ["borrow limit", "maximum books", "borrow count", "library rules"]
    },
    {
        "question": "Is there a waiting list for popular books?",
        "answer": "Yes, if a book is currently checked out, you can place a hold on it, and you'll be notified when it's available.",
        "tags": ["waiting list", "popular books", "holds", "reservations"]
    },
    {
        "question": "Can I donate books to the library?",
        "answer": "Yes, the library accepts book donations. Please contact the library for guidelines on donations.",
        "tags": ["donate", "book donation", "give books", "library donations"]
    },
    {
        "question": "Does the library have a quiet study area?",
        "answer": "Yes, the library has dedicated quiet study areas for individual work.",
        "tags": ["study area", "quiet space", "individual work", "study room"]
    },
    {
        "question": "Can I host an event at the library?",
        "answer": "Yes, the library offers spaces that can be reserved for events. Please contact the library for availability and booking.",
        "tags": ["host event", "library space", "booking", "reservations", "community events"]
    },
    {
        "question": "How do I report a lost library card?",
        "answer": "If your library card is lost or stolen, please report it immediately at the library desk or through the online portal.",
        "tags": ["lost card", "library card", "report", "stolen card", "membership"]
    }
]

# Start testing
print('Testing Cases starting..........\n')

unseen = []
for entry in library_data:
    print(entry['question'])
    print('\n')

    # Assuming generate_response is available
    # Use the model to generate an answer (replace this with your actual function)
    text = generate_response(entry['question'].lower(), model, words, classes)

    # Format and display the answer using textwrap
    print(textwrap.fill(text, width=70))
    print('-------------------------------------')

    # If the answer is not satisfactory, add the question to unseen list
    if text == "I'm sorry, but I'm not sure how to respond to that.":
        unseen.append(entry['question'])

# Print all unseen questions for further review
print("\nUnseen Questions:")
for question in unseen:
    print(question)


Testing Cases starting..........

How do I borrow books from the library?


To borrow books, just show your library card at the desk, or use the
self-checkout machine available for your convenience.
-------------------------------------
How can I renew a book?


You can easily renew books through the library portal or by visiting
the library desk. Ensure the book isn’t reserved by another user
before renewing.
-------------------------------------
What are the library's working hours?


The library is open Monday to Friday from 8 AM to 8 PM and on Saturday
from 10 AM to 4 PM. We are closed on Sundays.
-------------------------------------
Does the library provide Wi-Fi access?


The library provides free Wi-Fi. Simply connect to 'Library_WiFi' and
start browsing. There is no password required.
-------------------------------------
What upcoming events are scheduled at the library?


The library hosts regular events such as book readings, workshops, and
community gatherings. Check the l

In [ ]:
unseen

[]